<a href="https://colab.research.google.com/github/2019zhou/helperFunction/blob/master/DataPreprocess.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Training using fairseq
- 一个使用bug数据集, fairseq框架跑的一个模型
- 基本上所有参数都还没有进行任何改动，使用bpe
- 使用了完全nlp方式来训练code数据
- 主要目的在于查看这样训练的可行性

## 下载两个对应的数据集
- src-all.txt
- tgt-all.txt
- 注意使用 -q 不然输出结果太大，colab会有报错


In [1]:
%%bash
wget -q https://zenodo.org/record/4067881/files/src-all.txt

In [2]:
%%bash
wget -q https://zenodo.org/record/4067881/files/tgt-all.txt

In [4]:
# 把数据拷贝到google云盘中
!cp '/content/src-all.txt' -r '/content/drive/My Drive/bugData/'

In [ ]:
# 如果上一次已经拷贝到了google drive中，这次只需要从google drive中把数据拷贝出来就可以了
!cp '/content/drive/My Drive/bugData/' -r '/content/src-all.txt'

## Explore the dataset

In [ ]:
%%bash
head -n 50 src-all.txt

void cpuidle_enter_freeze ( void ) { struct cpuidle_device * dev = __this_cpu_read ( cpuidle_devices ) ; struct cpuidle_driver * drv = cpuidle_get_cpu_driver ( dev ) ; int index ; if ( cpuidle_not_available ( drv , dev ) ) goto fallback ; index = cpuidle_find_deepest_state ( drv , dev , true ) ; if ( index >= 0 ) { enter_freeze_proper ( drv , dev , index ) ; local_irq_enable ( ) ; return ; } index = cpuidle_find_deepest_state ( drv , dev , false ) ; if ( index >= 0 ) { cpuidle_enter ( drv , dev , index ) ; return ; } fallback : arch_cpu_idle ( ) ; }
int cpuidle_select ( struct cpuidle_driver * drv , struct cpuidle_device * dev ) { if ( cpuidle_not_available ( drv , dev ) ) return - ENODEV ; return cpuidle_curr_governor -> select ( drv , dev ) ; }
static bool cpuidle_not_available ( struct cpuidle_driver * drv , struct cpuidle_device * dev ) { return off || ! initialized || ! drv || ! dev || ! dev -> enabled ; }
static int cpuidle_find_deepest_state ( struct cpuidle_driver * drv , struc

In [ ]:
%%bash 
head -n 50 tgt-all.txt

int cpuidle_enter_freeze ( struct cpuidle_driver * drv , struct cpuidle_device * dev ) { int index ; index = find_deepest_state ( drv , dev , true ) ; if ( index >= 0 ) enter_freeze_proper ( drv , dev , index ) ; return index ; }
int cpuidle_select ( struct cpuidle_driver * drv , struct cpuidle_device * dev ) { return cpuidle_curr_governor -> select ( drv , dev ) ; }
bool cpuidle_not_available ( struct cpuidle_driver * drv , struct cpuidle_device * dev ) { return off || ! initialized || ! drv || ! dev || ! dev -> enabled ; }
int cpuidle_find_deepest_state ( struct cpuidle_driver * drv , struct cpuidle_device * dev ) { return find_deepest_state ( drv , dev , false ) ; }
bool process_record_user ( uint16_t keycode , keyrecord_t * record ) { switch ( keycode ) { case LOWER : if ( record -> event . pressed ) { layer_on ( _LOWER ) ; update_tri_layer ( _LOWER , _RAISE , _ADJUST ) ; } else { layer_off ( _LOWER ) ; update_tri_layer ( _LOWER , _RAISE , _ADJUST ) ; } return false ; break ; case 

In [ ]:
print("The src.txt contains words, characters, lines ")
!wc -mlw src-all.txt
print("The tgt.txt contains words, characters, lines ")
!wc -mlw tgt-all.txt

The src.txt contains words, characters, lines 
    643695  356284731 2060915253 src-all.txt
The tgt.txt contains words, characters, lines 
    643695  360890953 2091037112 tgt-all.txt


## 进行BPE preprocess

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
%%bash


awk: cannot open /train.tags..tok.src-all.txt (No such file or directory)
awk: cannot open /train.tags..tok.src-all.txt (No such file or directory)
awk: cannot open /train.tags..tok.tgt-all.txt (No such file or directory)
awk: cannot open /train.tags..tok.tgt-all.txt (No such file or directory)


In [ ]:
%%bash
mkdir -p gpt2_bpe
wget -O gpt2_bpe/encoder.json https://dl.fbaipublicfiles.com/fairseq/gpt2_bpe/encoder.json
wget -O gpt2_bpe/vocab.bpe https://dl.fbaipublicfiles.com/fairseq/gpt2_bpe/vocab.bpe

--2020-11-04 15:26:04--  https://dl.fbaipublicfiles.com/fairseq/gpt2_bpe/encoder.json
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 104.22.74.142, 104.22.75.142, 172.67.9.4, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|104.22.74.142|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1042301 (1018K) [text/plain]
Saving to: ‘gpt2_bpe/encoder.json’

     0K .......... .......... .......... .......... ..........  4%  803K 1s
    50K .......... .......... .......... .......... ..........  9%  628K 1s
   100K .......... .......... .......... .......... .......... 14% 1.01M 1s
   150K .......... .......... .......... .......... .......... 19% 92.5M 1s
   200K .......... .......... .......... .......... .......... 24% 88.7M 1s
   250K .......... .......... .......... .......... .......... 29%  854K 1s
   300K .......... .......... .......... .......... .......... 34% 30.4M 0s
   350K .......... .......... .......... .......... ........

In [ ]:
python -m examples.roberta.multiprocessing_bpe_encoder \
        --encoder-json gpt2_bpe/encoder.json \
        --vocab-bpe gpt2_bpe/vocab.bpe \
        --inputs src-all.raw \
        --outputs src.bpe \
        --keep-empty \
        --workers 60;

SyntaxError: ignored